### VIDEO SEGMENTATION BY CUSTOM ARCHITECTURE

**Highlights:**
 <br> 1. Semantic Segmentation
 <br> 2. Image normalization 
 <br> 3. Obtain Frames from Video and bundle back the segmented frames
 <br> 4. Early stopping and dropout to avoid model overfitting 
 <br> 5. Works for live cameras

Note: The code is just for conceptual implementations and did not utilise the actual organizational dataset. Hence, the training is very limited as the extensive training dataset is not available. Hence may not be precise for random footages

In [ ]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator,load_img, img_to_array,array_to_img
from tensorflow.keras.layers import Conv2D, SeparableConv2D, Dropout,BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
def Fetch_Data(dataset_path,dimension):
    images = glob(os.path.join(dataset_path, "original_images/*.png"))
    labels = glob(os.path.join(dataset_path, "label_images/*.png"))
    image = []
    label = []
    for i in range (len(images)):
        inp = plt.imread( images[i])
        out = plt.imread( labels[i])        
        inp = cv2.resize(inp, (dimension, dimension)) 
        out = cv2.resize(out, (dimension, dimension))
        image.append(inp)
        label.append(out)   
    return image,label

In [ ]:
image,label = Fetch_Data("Frame Segmentation",256)

In [ ]:
def SepConv2D(input, num_filters):
    conv = SeparableConv2D(num_filters, 3, padding="same")(input)
    conv = BatchNormalization()(conv)
    conv = Activation("relu")(conv)

    conv = SeparableConv2D(num_filters, 3, padding="same")(conv)
    conv = BatchNormalization()(conv)
    conv = Activation("relu")(conv)

    return conv

def en(input, filters):
    conv = SepConv2D(input, filters)
    Agr = MaxPool2D((2, 2))(conv)
    Agr = Dropout(0.05)(Agr)
    return conv, Agr

def de(input, skip_features, filters):
    conv = Conv2DTranspose(filters, (2, 2), strides=2, padding="same")(input)
    conv = Concatenate()([conv, skip_features])
    Agr = Dropout(0.07)(conv)
    conv = SepConv2D(Agr, filters)
    return conv

def seg_deconv_model(input_shape,classes):
    inputs = Input(input_shape)

    cn1, Agr1 = en(inputs, 16)
    cn2, Agr2 = en(Agr1, 32)
    cn3, Agr3 = en(Agr2, 64)
    cn4, Agr4 = en(Agr3, 128)

    bottleneck = SepConv2D(Agr4, 256)

    decn1 = de(bottleneck, cn4, 128)
    decn2 = de(decn1, cn3, 64)
    decn3 = de(decn2, cn2, 32)
    decn4 = de(decn3, cn1, 16)

    outputs = SeparableConv2D(classes, 1, padding="same", activation="sigmoid")(decn4)

    model = Model(inputs, outputs)
    return model

In [ ]:
inputsize = (256,256,3)
neural_system = seg_deconv_model(inputsize, 3)

neural_system.compile(loss="binary_crossentropy",optimizer= 'Adam',metrics = ['accuracy'])
callbacks = [
        ModelCheckpoint("CustomArch.h5", monitor="val_loss", verbose=1),
        ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
        EarlyStopping(monitor="val_loss", patience=10)
        ]

In [ ]:
neural_system.fit(np.array(image), np.array(label), epochs = 200, verbose = 1)

In [ ]:
import cv2
import math

imagesFolder = "images"
cap = cv2.VideoCapture("input_video_1.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# initialize the FourCC and a video writer object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('output_1.mp4', fourcc, 5, (frame_width, frame_height))

while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    
      
    if (ret != True):
        break
    for i in range(10):
        filename = 'D:/images' + "//" + '%s.png' % i
        cv2.imwrite(filename, frame)
        
    path = glob.glob('D:/images/*.png')
    inp = []
    for i in range (len(path)):
        x = plt.imread( path[i])   
        x = cv2.resize(x, (256, 256)) 
        inp.append(x)  
  
    inproc = np.array(inp)
    labelled_frame = neural_system.predict(inproc)[1]
    inp = []
    min_val,max_val=labelled_frame.min(),labelled_frame.max()  
    labelled_frame = 255.0*(labelled_frame - min_val)/(max_val - min_val)
    labelled_frame = labelled_frame.astype(np.uint8)
    labelled_frame = cv2.resize(labelled_frame, (frame_width, frame_height))
#         filename = imagesFolder + "/labelled_frame-" +  str(int(frameId)) + ".png"
#         cv2.imwrite(filename, labelled_frame)
    output.write(labelled_frame)

cap.release()
output.release()
print ("Done!")

### THE END